In [1]:
import json
from os.path import isfile, join
import pathlib

from PIL import Image

IMAGES_DIR = join('.', 'imgs')
IMAGES_DIR_RAW = join(IMAGES_DIR, '00_raw')
IMAGES_DIR_FACES = join(IMAGES_DIR, '00_raw_faces')

WEB_DIR = join('..', 'cordiais-web', 'public')
WEB_DIR_FACES = join(WEB_DIR, 'imgs', 'faces')
WEB_DIR_DATA = join(WEB_DIR, 'data')
WEB_DATA_FILE = join(WEB_DIR_DATA, 'obras.json')

obras_web = {}

with open(WEB_DATA_FILE, 'r') as in_json:
    obras_web = json.load(in_json)


In [2]:
def crop_face(img, face_rect):
    iwidth, iheight = img.size

    face_left = face_rect['left'] * iwidth
    face_top = face_rect['top'] * iheight
    face_width = face_rect['width'] * iwidth
    face_height = face_rect['height'] * iheight

    face_center_x = face_left + 0.5 * face_width
    face_center_y = face_top + 0.475 * face_height
    face_dim = max (face_width, face_height)

    crop_margin = 0.666
    crop_left = face_center_x - crop_margin * face_dim
    crop_right = face_center_x + crop_margin * face_dim
    crop_top = face_center_y - crop_margin * face_dim
    crop_bottom = face_center_y + crop_margin * face_dim

    return img.crop((crop_left, crop_top, crop_right, crop_bottom))


In [3]:
def resize_img(img, max_dim):
    img.thumbnail((max_dim, max_dim), Image.ANTIALIAS)
    return img

In [4]:
pathlib.Path(IMAGES_DIR_FACES).mkdir(parents=True, exist_ok=True)
pathlib.Path(WEB_DIR_FACES).mkdir(parents=True, exist_ok=True)

for o_slug in obras_web:
    o = obras_web[o_slug]

    o_img_file = join(IMAGES_DIR_RAW, o['img'].replace('_web', '_raw'))
    o_face_file = join(IMAGES_DIR_FACES, o['img'].replace('_web', '_raw'))
    o_face_file_web = join(WEB_DIR_FACES, o['img'])

    have_both_faces = isfile(o_face_file) and isfile(o_face_file_web)

    if isfile(o_img_file) and 'face_rectangle' in o and not have_both_faces:
        img = Image.open(o_img_file).convert('RGB')
        face = crop_face(img, o['face_rectangle'])

        if not isfile(o_face_file):
            face.save(o_face_file, quality=90, optimize=True, progressive=True)

        if not isfile(o_face_file_web):
            face_web = resize_img(face.copy(), 512)
            face_web.save(o_face_file_web, quality=90, optimize=True, progressive=True)
